Libraries

In [1]:
import os, pickle, numpy as np, pandas as pd
from GloVex.evaluate_personalised import survey_reader
from scipy.stats import pearsonr
cwd = os.getcwd()

Get the surprises of the users in the survey

In [2]:
survey_reader_obj = survey_reader()
food_cuisine_survey_fn = cwd + '/personalized-surprise/Food and Cuisine Preferences Survey (Responses) - Form Responses 2'
fam_cat_sorted = ['mexican', 'chinese', 'modern', 'greek', 'indian', 'thai', 'italian']
users_fam_dir, familiar_cols, users_cuisine_pref, preference_cols, \
users_knowledge_ingredient, knowledge_ingredient_cols, users_knowledge_cuisine, cuisine_knowledge_cols, \
users_surp_ratings, surprise_rating_cols, users_surp_pref, surprise_preference_cols = \
	survey_reader_obj.read_survey(food_cuisine_survey_fn, fam_cat_sorted)

GloVex/evaluate_personalised.py:313: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  users_df['col_list'] = (np.array(users_df.values.tolist()) / scale).tolist()


Divide the users into foodies and novices using the median of sum of familiarities

In [3]:
# Array holding all of the sums
sum_arr = []
# Iterate over the users' familiarities
for user_idx in sorted(users_fam_dir.keys()):
	# Get the familiarity scores
	each_fam = users_fam_dir[user_idx]
	# Calculate the sum of familiarity for each user
	sum_fam = sum(each_fam)
	# Append the sum of familiarity of each user to the sum array
	sum_arr.append(sum_fam)
median_fam = np.median(sum_arr)
# Split the users into foodeis and novices using the median
foodie_fam_dict = {}
foodie_surp_ratings_dict = {}
novice_fam_dict = {}
novice_surp_ratings_dict = {}
# Iterate over the users' familiarities
for user_idx in sorted(users_fam_dir.keys()):
	# Get the familiarity and surprise scores
	each_fam = users_fam_dir[user_idx]
	each_surp = users_surp_ratings[user_idx]
	# Calculate the sum of familiarity for each user
	sum_fam = sum(each_fam)
	if sum_fam > median_fam:
		# print 'above median', each_fam
		foodie_fam_dict[user_idx] = each_fam
		foodie_surp_ratings_dict[user_idx] = each_surp
	else:
		# print 'below median', each_fam
		novice_fam_dict[user_idx] = each_fam
		novice_surp_ratings_dict[user_idx] = each_surp
# List the foodies and novices in two separate lists
foodie_users = sorted(foodie_fam_dict.keys())
novice_users = sorted(novice_fam_dict.keys())

Get cuisine softmax probability

In [4]:
# 73106,mexican
# 73107,thai
# 73108,modern
# 73109,modern
# 73110,chinese
# 73111,indian
# 73112,italian
# 73113,modern
# 73114,greek
# 73115,mexican
# 73116,greek
# 73117,thai
# 73118,modern
# 73119,italian
# 73120,chinese
# 73121,indian
fam_cat_sorted = ['mexican', 'chinese', 'modern', 'greek', 'indian', 'thai', 'italian']
recipe_cuisines = \
{73106: 'mexican', 73107: 'thai', 73108: 'modern', 73109: 'modern', 73110: 'chinese', 73111: 'indian', 73112: 'italian', 73113: 'modern',
 73114: 'greek', 73115: 'mexican', 73116: 'greek', 73117: 'thai', 73118: 'modern', 73119: 'italian', 73120: 'chinese', 73121: 'indian'}
cuisine_softmax = {}
for each_recipe in sorted(recipe_cuisines):
	# print each_recipe, recipe_cuisines[each_recipe]
	cuisine_softmax[each_recipe] = []
	for each_cuisine in fam_cat_sorted:
		# print 'each_cuisine', each_cuisine
		if recipe_cuisines[each_recipe] == each_cuisine:
			cuisine_softmax[each_recipe].append(0.2)
		else:
			cuisine_softmax[each_recipe].append(0)
cuisine_softmax_cols = [each_cuisine + '_softmax' for each_cuisine in fam_cat_sorted]

Get the personalized surprise estimates by the model

In [5]:
familiarity_measure = 'recipes_self_reported'
# familiarity_measure = 'recipes_knowledge'
# familiarity_measure = 'recipes_averaged'
user_suprise_estimates_pickle_fn = cwd + '/GloVex/results/' + familiarity_measure + '/user_suprise_estimates.pickle'
user_suprise_estimates_dict = pickle.load(open(user_suprise_estimates_pickle_fn,'rb'))

Get the personalized surprise estimates (95th percentile and max score)

In [6]:
# Initialize surprise estimates for 95th percentile and max score
personalized_surp_estimates_90perc_dict = {}
personalized_surp_estimates_95perc_dict = {}
personalized_surp_estimates_max_dict = {}
# Iterate over the user surprise estimeates dict
for each_user in sorted(user_suprise_estimates_dict.keys()):
	# print 'each_user', each_user
	# Initialize the inner dicts for 90th and 95th percentiles and max score
	personalized_surp_estimates_90perc_dict[each_user] = {}
	personalized_surp_estimates_95perc_dict[each_user] = {}
	personalized_surp_estimates_max_dict[each_user] = {}
	# Iterate over the surprises for each user's surprise estimates
	for each_recipe in sorted(user_suprise_estimates_dict[each_user]['recipes_surp']):
		# print 'each_recipe', each_recipe
		# Get the 90th and 95th percentiles and max
		user_suprise_90th_percentile = user_suprise_estimates_dict[each_user]['recipes_surp'][each_recipe]['90th_percentile']
		user_suprise_95th_percentile = user_suprise_estimates_dict[each_user]['recipes_surp'][each_recipe]['95th_percentile']
		user_suprise_max = max([each_ingr_comb[2] 
			for each_ingr_comb in user_suprise_estimates_dict[each_user]['recipes_surp'][each_recipe]['surprise_cuisine']])
		# Add the user's surprise estimates to the dict
		personalized_surp_estimates_90perc_dict[each_user][each_recipe] = user_suprise_90th_percentile
		personalized_surp_estimates_95perc_dict[each_user][each_recipe] = user_suprise_95th_percentile
		personalized_surp_estimates_max_dict[each_user][each_recipe] = user_suprise_max

Get the oracle surprise estimates by the model

In [7]:
oracle_suprise_pickle_fn = cwd + '/GloVex/results/recipes_oracle/oracle_suprise.pickle'
oracle_suprise_dict = pickle.load(open(oracle_suprise_pickle_fn, 'rb'))

Get the oracle's surprise estimates (90th and 95th percentiles and max score)

In [8]:
# Initialize arrays
oracle_surp_estimates_90perc_dict = {}
oracle_surp_estimates_95perc_dict = {}
oracle_surp_estimates_max_dict = {}
observed_surp_estimates_90perc_dict = {}
observed_surp_estimates_95perc_dict = {}
observed_surp_estimates_max_dict = {}
# Iterate over the dict
for each_recipe in sorted(oracle_suprise_dict):
	# print 'each_recipe', each_recipe
	# Get all of the ingredients combinations' surprise estimates
	oracle_ingr_surp_estimates = [each_ingr_comb[2] for each_ingr_comb in oracle_suprise_dict[each_recipe]['surprise_cuisine']]
	observed_ingr_surp_estimates = [each_ingr_comb[2] for each_ingr_comb in oracle_suprise_dict[each_recipe]['observed_surprise_cuisine']]
	# Add the 90th and 95th_percentiles and max surprise estimates to the dict
	oracle_surp_estimates_90perc_dict[each_recipe] = oracle_suprise_dict[each_recipe]['90th_percentile']
	oracle_surp_estimates_95perc_dict[each_recipe] = oracle_suprise_dict[each_recipe]['95th_percentile']
	oracle_surp_estimates_max_dict[each_recipe] = max(oracle_ingr_surp_estimates)
	observed_surp_estimates_90perc_dict[each_recipe] = oracle_suprise_dict[each_recipe]['observed_90th_percentile']
	observed_surp_estimates_95perc_dict[each_recipe] = oracle_suprise_dict[each_recipe]['observed_95th_percentile']
	observed_surp_estimates_max_dict[each_recipe] = max(observed_ingr_surp_estimates)

Compile data

For all users

In [9]:
# Get recipes' and users' keys
users_keys = sorted(users_fam_dir.keys())
recipes_keys = sorted(oracle_suprise_dict.keys())
# Initialize the array that will hold the users' input for training using the header
user_input_train_arr = [['User ID', 'Recipe ID'] + familiar_cols + preference_cols + cuisine_knowledge_cols + cuisine_softmax_cols + \
	['observed_surp_estimates_90perc', 'observed_surp_estimates_95perc', 'observed_surp_estimates_max',
	 'oracle_surp_estimates_90perc', 'oracle_surp_estimates_95perc', 'oracle_surp_estimates_max',
	 'personalized_surp_estimates_90perc', 'personalized_surp_estimates_95perc', 'personalized_surp_estimates_max',
	 'users_surp_pref', 'users_surp_ratings']]
# Iterate over the users' keys
for each_user in users_keys:
	# print 'each_user', each_user
	for recipe_idx, each_recipe in enumerate(recipes_keys):
		# print recipe_idx, each_recipe
		# Get the users' familiarity, users' cuisine preference, users' knowledge, recipe's softmax probability
		user_input_train_arr.append(
			[each_user, each_recipe] + \
			users_fam_dir[each_user] + \
			users_cuisine_pref[each_user] + \
			users_knowledge_cuisine[each_user] + \
			cuisine_softmax[each_recipe] + \
			[
				observed_surp_estimates_90perc_dict[each_recipe], + \
				observed_surp_estimates_95perc_dict[each_recipe], + \
				observed_surp_estimates_max_dict[each_recipe], + \
				oracle_surp_estimates_90perc_dict[each_recipe], + \
				oracle_surp_estimates_95perc_dict[each_recipe], + \
				oracle_surp_estimates_max_dict[each_recipe], + \
				personalized_surp_estimates_90perc_dict[each_user][each_recipe], + \
				personalized_surp_estimates_95perc_dict[each_user][each_recipe], + \
				personalized_surp_estimates_max_dict[each_user][each_recipe], + \
				users_surp_pref[each_user][recipe_idx], + \
				users_surp_ratings[each_user][recipe_idx]
			]
		)

For foodies

In [10]:
# Initialize the array that will hold the users' input for training using the header
foodies_input_train_arr = [['User ID', 'Recipe ID'] + familiar_cols + preference_cols + cuisine_knowledge_cols + cuisine_softmax_cols + \
	['observed_surp_estimates_90perc', 'observed_surp_estimates_95perc', 'observed_surp_estimates_max',
	 'oracle_surp_estimates_90perc', 'oracle_surp_estimates_95perc', 'oracle_surp_estimates_max',
	 'personalized_surp_estimates_90perc', 'personalized_surp_estimates_95perc', 'personalized_surp_estimates_max',
	 'users_surp_pref', 'users_surp_ratings']]
# Iterate over the users' keys
for each_user in foodie_users:
	# print 'each_user', each_user
	for recipe_idx, each_recipe in enumerate(recipes_keys):
		# print recipe_idx, each_recipe
		# Get the users' familiarity, users' cuisine preference, users' knowledge, recipe's softmax probability
		foodies_input_train_arr.append(
			[each_user, each_recipe] + \
			users_fam_dir[each_user] + \
			users_cuisine_pref[each_user] + \
			users_knowledge_cuisine[each_user] + \
			cuisine_softmax[each_recipe] + \
			[
				observed_surp_estimates_90perc_dict[each_recipe], + \
				observed_surp_estimates_95perc_dict[each_recipe], + \
				observed_surp_estimates_max_dict[each_recipe], + \
				oracle_surp_estimates_90perc_dict[each_recipe], + \
				oracle_surp_estimates_95perc_dict[each_recipe], + \
				oracle_surp_estimates_max_dict[each_recipe], + \
				personalized_surp_estimates_90perc_dict[each_user][each_recipe], + \
				personalized_surp_estimates_95perc_dict[each_user][each_recipe], + \
				personalized_surp_estimates_max_dict[each_user][each_recipe], + \
				users_surp_pref[each_user][recipe_idx], + \
				users_surp_ratings[each_user][recipe_idx]
			]
		)

For novices

In [11]:
# Initialize the array that will hold the users' input for training using the header
novices_input_train_arr = [['User ID', 'Recipe ID'] + familiar_cols + preference_cols + cuisine_knowledge_cols + cuisine_softmax_cols + \
	['observed_surp_estimates_90perc', 'observed_surp_estimates_95perc', 'observed_surp_estimates_max',
	 'oracle_surp_estimates_90perc', 'oracle_surp_estimates_95perc', 'oracle_surp_estimates_max',
	 'personalized_surp_estimates_90perc', 'personalized_surp_estimates_95perc', 'personalized_surp_estimates_max',
	 'users_surp_pref', 'users_surp_ratings']]
# Iterate over the users' keys
for each_user in novice_users:
	# print 'each_user', each_user
	for recipe_idx, each_recipe in enumerate(recipes_keys):
		# print recipe_idx, each_recipe
		# Get the users' familiarity, users' cuisine preference, users' knowledge, recipe's softmax probability
		novices_input_train_arr.append(
			[each_user, each_recipe] + \
			users_fam_dir[each_user] + \
			users_cuisine_pref[each_user] + \
			users_knowledge_cuisine[each_user] + \
			cuisine_softmax[each_recipe] + \
			[
				observed_surp_estimates_90perc_dict[each_recipe], + \
				observed_surp_estimates_95perc_dict[each_recipe], + \
				observed_surp_estimates_max_dict[each_recipe], + \
				oracle_surp_estimates_90perc_dict[each_recipe], + \
				oracle_surp_estimates_95perc_dict[each_recipe], + \
				oracle_surp_estimates_max_dict[each_recipe], + \
				personalized_surp_estimates_90perc_dict[each_user][each_recipe], + \
				personalized_surp_estimates_95perc_dict[each_user][each_recipe], + \
				personalized_surp_estimates_max_dict[each_user][each_recipe], + \
				users_surp_pref[each_user][recipe_idx], + \
				users_surp_ratings[each_user][recipe_idx]
			]
		)

Save the training input

In [12]:
# Save all users input
user_input_train_df = pd.DataFrame(user_input_train_arr)
user_input_train_fn = cwd + '/qchef_input_training/users_input_train_' + familiarity_measure + '.csv'
user_input_train_df.to_csv(user_input_train_fn, index=False, header=None)

In [13]:
# Save foodies input
foodie_input_train_df = pd.DataFrame(foodies_input_train_arr)
foodie_input_train_fn = cwd + '/qchef_input_training/foodies_input_train_' + familiarity_measure + '.csv'
foodie_input_train_df.to_csv(foodie_input_train_fn, index=False, header=None)

In [14]:
# Save novices input
novice_input_train_df = pd.DataFrame(novices_input_train_arr)
novice_input_train_fn = cwd + '/qchef_input_training/novices_input_train_' + familiarity_measure + '.csv'
novice_input_train_df.to_csv(novice_input_train_fn, index=False, header=None)